# 30 Forecast Horizon 

#### This notebook has the models forecasting only for a 30day forecast with fine tuning

In [1]:
#Import the neccesary libraries
import warnings
warnings.filterwarnings('ignore')
import random

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.gridspec as gr
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from tabulate import tabulate
import holidays

In [2]:
df = pd.read_csv("processed.csv", parse_dates=['date'], index_col='date')
df =df.asfreq('D')
df =df[["quantity"]]
df.head()

In [3]:
data = df.copy()
data["unique_id"]=1.0
data["ds"] = data.index
data.rename(columns={"quantity":"y"},inplace=True)
data.head()

In [4]:
# Using Nxitla libraries
#Data Split
# Calculate the index for the split
split_index = int(0.8 * len(data))

# Split the data
Y_train_df = data.iloc[:split_index]
Y_test_df = data.iloc[split_index:]   # Test data for January 2012

horizon = len(Y_test_df)

In [5]:
ml_data = data.copy()
train_df = ml_data [:-30]
test_df = ml_data [-30:]

In [8]:
import holidays
# Function to create date features
def create_date_features(data):
    data["month"] = data.index.month
    data["day_of_month"] = data.index.day
    data["is_month_start"] = data.index.is_month_start.astype(int)
    data["is_month_end"] = data.index.is_month_end.astype(int)
    data["day_of_year"] = data.index.dayofyear
    data["week_of_year"] = data.index.isocalendar().week
    data["day_of_week"] = data.index.dayofweek + 1
    data["year"] = data.index.year
    data["is_weekend"] = (data.index.weekday >= 5).astype(int)
    data['is_spring'] = data['month'].isin([3, 4, 5]).astype(int)
    data['is_summer'] = data['month'].isin([6, 7, 8]).astype(int)
    data['is_fall'] = data['month'].isin([9, 10, 11]).astype(int)
    data['is_winter'] = data['month'].isin([12, 1, 2]).astype(int)
    data['sin_day'] = np.sin(2 * np.pi * data.index.dayofweek / 7)
    data['cos_day'] = np.cos(2 * np.pi * data.index.dayofweek / 7)
    return data

# Add holiday feature
uk_holidays = holidays.CountryHoliday('UK')


In [7]:
import pandas as pd
import numpy as np
from statsforecast.models import AutoARIMA, AutoETS, AutoCES, AutoTheta, SimpleExponentialSmoothingOptimized
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error
import matplotlib.pyplot as plt

# Assuming train_df, test_df, and ml_data are already defined in your environment

# Extract the 'y' series from train and test dataframes
train_series = train_df['y'].values
test_series = test_df['y'].values

# Define the models
models = {
    'AutoARIMA': AutoARIMA(season_length=12),
    'AutoETS': AutoETS(model='ZZZ', season_length=12),
    'AutoCES': AutoCES(model='Z', season_length=12),
    'AutoTheta': AutoTheta(season_length=12),
    'SESOpt': SimpleExponentialSmoothingOptimized()
}

# Initialize lists to store results
mape_scores = []
mae_scores = []

# Train and evaluate each model
for name, model in models.items():
    model = model.fit(y=train_series)
    forecasts = model.predict(h=len(test_series))
    
    # Calculate MAE and MAPE
    mae = mean_absolute_error(test_series, forecasts['mean'])
    mape = mean_absolute_percentage_error(test_series, forecasts['mean'])
    
    mae_scores.append((name, mae))
    mape_scores.append((name, mape))
    
    print(f"{name} - MAE: {mae:.4f}, MAPE: {mape:.4%}")

# Create a DataFrame to display the results
results_df = pd.DataFrame(mae_scores, columns=['Model', 'MAE']).merge(
    pd.DataFrame(mape_scores, columns=['Model', 'MAPE']), on='Model'
)

# Display the results DataFrame
print(results_df)

# Select the best model based on MAPE
best_model_name, best_mape = min(mape_scores, key=lambda x: x[1])
best_model = models[best_model_name]

print(f"\nThe best model is {best_model_name} with a MAPE of {best_mape:.4%}")

# Train the best model on the entire dataset
best_model = best_model.fit(y=ml_data['y'].values)

# Predict the next 30 days
future_forecasts = best_model.predict(h=30)

# Combine the historical data with the forecast data for plotting
combined_df = ml_data.copy()
future_df = pd.DataFrame({
    'ds': pd.date_range(start=ml_data['ds'].max() + pd.Timedelta(days=1), periods=30, freq='D'),
    'y': np.nan
})
combined_df = pd.concat([combined_df, future_df], ignore_index=True)
combined_df['forecast'] = np.nan
combined_df.loc[ml_data.shape[0]:, 'forecast'] = future_forecasts['mean']

# Plot actual vs predicted values for the historical period and future forecast
plt.figure(figsize=(14, 7))

# Plot the entire dataset
plt.plot(combined_df['ds'], combined_df['y'], label='Actual', marker='o')

# Plot the future forecasts
plt.plot(
    combined_df['ds'],
    combined_df['forecast'],
    label=f'Predicted - {best_model_name} (Next 30 Days)',
    marker='x',
)

plt.xlabel('Date')
plt.ylabel('Value')
plt.title(f'Actual vs Predicted Values using {best_model_name}')
plt.legend()
plt.grid(True)
plt.show()


# Machine Learning

In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
import holidays

# Function to create date features
def create_date_features(data):
    data["month"] = data.index.month
    data["day_of_month"] = data.index.day
    data["is_month_start"] = data.index.is_month_start.astype(int)
    data["is_month_end"] = data.index.is_month_end.astype(int)
    data["day_of_year"] = data.index.dayofyear
    data["week_of_year"] = data.index.isocalendar().week
    data["day_of_week"] = data.index.dayofweek + 1
    data["year"] = data.index.year
    data["is_weekend"] = (data.index.weekday >= 5).astype(int)
    data['is_spring'] = data['month'].isin([3, 4, 5]).astype(int)
    data['is_summer'] = data['month'].isin([6, 7, 8]).astype(int)
    data['is_fall'] = data['month'].isin([9, 10, 11]).astype(int)
    data['is_winter'] = data['month'].isin([12, 1, 2]).astype(int)
    data['sin_day'] = np.sin(2 * np.pi * data.index.dayofweek / 7)
    data['cos_day'] = np.cos(2 * np.pi * data.index.dayofweek / 7)
    return data

# Add holiday feature
uk_holidays = holidays.CountryHoliday('UK')

# Create date features
rml_data = create_date_features(data.copy())
rml_data['is_public_holiday'] = rml_data.index.to_series().apply(lambda date: 1 if date in uk_holidays else 0)

# Define Features and Models
date_features = ['month', 'day_of_month', 'is_month_start', 'is_month_end',
                 'day_of_year', 'week_of_year', 'day_of_week', 'year',
                 'is_weekend', 'is_spring', 'is_summer', 'is_fall', 'is_winter',
                 'sin_day', 'cos_day', 'is_public_holiday']

# Prepare training data
def create_lagged_features(data, lags):
    for lag in lags:
        data[f'lag_{lag}'] = data['y'].shift(lag)
    return data

lags = [7, 14, 30]
rml_data = create_lagged_features(rml_data, lags).dropna()

X = rml_data[date_features + [f'lag_{lag}' for lag in lags]]
y = rml_data['y']

# Split data into train and test sets
train_X = X[:-30]
train_y = y[:-30]
test_X = X[-30:]
test_y = y[-30:]

# Define parameter grids for each model
param_grids = {
    'RandomForest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30],
        'min_samples_split': [2, 5, 10]
    },
    'XGBoost': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 6, 9],
        'learning_rate': [0.01, 0.1, 0.2]
    },
    'LightGBM': {
        'n_estimators': [100, 200, 300],
        'num_leaves': [31, 40, 50],
        'learning_rate': [0.01, 0.1, 0.2]
    }
}

# Initialize lists to store results
mae_scores = {}
mape_scores = {}
predictions = {}
best_params = {}

def smape(a, f):
    return 100 * np.mean(2 * np.abs(f - a) / (np.abs(a) + np.abs(f)))

# Train and predict using each model with grid search
for model_name, model in {'RandomForest': RandomForestRegressor(), 
                          'XGBoost': XGBRegressor(objective='reg:squarederror'), 
                          'LightGBM': lgb.LGBMRegressor()}.items():
    grid_search = GridSearchCV(model, param_grids[model_name], cv=3, scoring='neg_mean_absolute_percentage_error')
    grid_search.fit(train_X, train_y)
    best_params[model_name] = grid_search.best_params_
    predictions[model_name] = grid_search.predict(test_X)
    
    # Calculate MAE and MAPE
    mae = mean_absolute_error(test_y, predictions[model_name])
    mape = mean_absolute_percentage_error(test_y, predictions[model_name])
    
    mae_scores[model_name] = mae
    mape_scores[model_name] = mape
    
    print(f"{model_name} - Best Params: {best_params[model_name]}, MAE: {mae:.4f}, MAPE: {mape:.4%}")

# Create a DataFrame to display the results
results_df = pd.DataFrame(mae_scores.items(), columns=['Model', 'MAE']).merge(
    pd.DataFrame(mape_scores.items(), columns=['Model', 'MAPE']), on='Model'
)

# Display the results DataFrame
print(results_df)

# Select the best model based on MAPE
best_model_name = min(mape_scores, key=mape_scores.get)
print(f"\nThe best model is {best_model_name} with a MAPE of {mape_scores[best_model_name]:.4%}")

# Retrain the best model on the entire dataset with best parameters
best_model = {'RandomForest': RandomForestRegressor(**best_params['RandomForest']),
              'XGBoost': XGBRegressor(objective='reg:squarederror', **best_params['XGBoost']),
              'LightGBM': lgb.LGBMRegressor(**best_params['LightGBM'])}[best_model_name]

best_model.fit(X, y)

# Predict the next 30 days
future_dates = pd.date_range(start=X.index[-1] + pd.Timedelta(days=1), periods=30, freq='D')
future_data = pd.DataFrame(index=future_dates)
future_data = create_date_features(future_data)
future_data['is_public_holiday'] = future_data.index.to_series().apply(lambda date: 1 if date in uk_holidays else 0)

# Initialize the lagged features with previous values
for lag in lags:
    future_data[f'lag_{lag}'] = np.nan

# Iterative prediction for the next 30 days
for i, date in enumerate(future_dates):
    if i == 0:
        lagged_features = [np.nan] * len(lags)  # No lagged values for the first date
    else:
        lagged_features = future_data.iloc[i-1][[f'lag_{lag}' for lag in lags]].values
    
    # Create the feature vector for the current date
    current_features = future_data.loc[date, date_features + [f'lag_{lag}' for lag in lags]].values.reshape(1, -1)
    future_data.at[date, 'Predicted'] = best_model.predict(current_features)[0]
    
    # Update lagged features for future iterations
    for j, lag in enumerate(lags):
        if date - pd.Timedelta(days=lag) in future_data.index:
            future_data.at[date, f'lag_{lag}'] = future_data.at[date, 'Predicted']

# Combine actual and predicted values
combined_df = pd.DataFrame({
    'Date': future_dates,
    'Predicted': future_data['Predicted']
}).set_index('Date')

# Plot actual vs predicted values
plt.figure(figsize=(14, 7))
plt.plot(test_y.index, test_y, label='Actual', marker='o')
plt.plot(test_y.index, predictions[best_model_name], label=f'Predicted - {best_model_name}', marker='x')
plt.plot(combined_df.index, combined_df['Predicted'], label='Future Predictions', marker='x')
plt.xlabel('Date')
plt.ylabel('Value')
plt.title('Actual vs Predicted Values')
plt.legend()
plt.grid(True)
plt.show()


# Deep learning

In [14]:
import pandas as pd
import numpy as np
from neuralforecast import NeuralForecast
from neuralforecast.models import NHITS, NBEATS, LSTM
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
import matplotlib.pyplot as plt

# Assuming you have defined train_df, test_df, and ml_data somewhere in your code

# Define the parameter grids for manual fine-tuning
nhits_param_grid = [
    {'h': 30, 'input_size': 30, 'max_steps': 50, 'learning_rate': 0.001},
    {'h': 30, 'input_size': 60, 'max_steps': 100, 'learning_rate': 0.01},
    {'h': 30, 'input_size': 90, 'max_steps': 150, 'learning_rate': 0.1},
]

nbeats_param_grid = [
    {'h': 30, 'input_size': 30, 'max_steps': 50, 'learning_rate': 0.001},
    {'h': 30, 'input_size': 60, 'max_steps': 100, 'learning_rate': 0.01},
    {'h': 30, 'input_size': 90, 'max_steps': 150, 'learning_rate': 0.1},
]

lstm_param_grid = [
    {'h': 30, 'input_size': 30, 'max_steps': 50, 'learning_rate': 0.001, 'encoder_n_layers': 2, 'encoder_hidden_size': 200},
    {'h': 30, 'input_size': 60, 'max_steps': 100, 'learning_rate': 0.01, 'encoder_n_layers': 2, 'encoder_hidden_size': 200},
    {'h': 30, 'input_size': 90, 'max_steps': 150, 'learning_rate': 0.1, 'encoder_n_layers': 3, 'encoder_hidden_size': 300},
]

# Initialize the models
nhits_model = NHITS(h=30, input_size=30)
nbeats_model = NBEATS(h=30, input_size=30)
lstm_model = LSTM(h=30, input_size=30)

# Train and evaluate each model with different hyperparameters
models = [nhits_model, nbeats_model, lstm_model]
param_grids = [nhits_param_grid, nbeats_param_grid, lstm_param_grid]
model_names = ['NHITS', 'NBEATS', 'LSTM']
mape_scores = []
mae_scores = []

for model, param_grid, name in zip(models, param_grids, model_names):
    for params in param_grid:
        if name == 'NHITS':
            model = NHITS(**params)
        elif name == 'NBEATS':
            model = NBEATS(**params)
        elif name == 'LSTM':
            model = LSTM(**params)
            
        nf = NeuralForecast(models=[model], freq='D')
        nf.fit(train_df)
        
        # Make predictions for the test set
        forecasts = nf.predict()
        forecasts = forecasts.set_index('ds')
        
        # Align forecast with test_df to calculate MAE and MAPE
        aligned_forecasts = forecasts[name].iloc[:len(test_df)].values
        
        mae = mean_absolute_error(test_df['y'], aligned_forecasts)
        mape = mean_absolute_percentage_error(test_df['y'], aligned_forecasts)
        
        mae_scores.append((name, mae, str(params)))
        mape_scores.append((name, mape, str(params)))
        
        print(f"{name} - MAE: {mae:.4f}, MAPE: {mape:.4%}")

# Create a DataFrame to display the results
results_df = pd.DataFrame(mae_scores, columns=['Model', 'MAE', 'Parameters']).merge(
    pd.DataFrame(mape_scores, columns=['Model', 'MAPE', 'Parameters']), on=['Model', 'Parameters']
)

# Display the results DataFrame
print(results_df)

# Select the best model based on MAPE
best_model_name, best_mape, best_params = min(mape_scores, key=lambda x: x[1])

if best_model_name == 'NHITS':
    best_model = NHITS(**eval(best_params))
elif best_model_name == 'NBEATS':
    best_model = NBEATS(**eval(best_params))
elif best_model_name == 'LSTM':
    best_model = LSTM(**eval(best_params))
    
print(f"The best model is {best_model_name} with a MAPE of {best_mape:.4%} and best parameters: {best_params}")

# Train the best model on the entire dataset
nf_best_model = NeuralForecast(models=[best_model], freq='D')
nf_best_model.fit(ml_data)

# Create a future dataframe for the next 30 days
future_df = nf_best_model.make_future_dataframe()

# Predict the next 30 days
future_forecasts = nf_best_model.predict()
future_forecasts = future_forecasts.set_index('ds')

# Combine the historical data with the forecast data for plotting
combined_df = pd.concat([ml_data, future_df], ignore_index=True)
combined_df['forecast'] = np.nan
combined_df.loc[ml_data.shape[0]:, 'forecast'] = future_forecasts[best_model_name].values

# Plot actual vs predicted values for the historical period and future forecast
plt.figure(figsize=(14, 7))

# Plot the entire dataset
plt.plot(combined_df['ds'], combined_df['y'], label='Actual', marker='o')

# Plot the future forecasts
plt.plot(
    combined_df['ds'],
    combined_df['forecast'],
    label=f'Predicted - {best_model_name} (Next 30 Days)',
    marker='x',
)

plt.xlabel('Date')
plt.ylabel('Value')
plt.title(f'Actual vs Predicted Values using {best_model_name}')
plt.legend()
plt.grid(True)
plt.show()

print(f"The best model is {best_model_name} with a MAPE of {best_mape:.4%} and best parameters: {best_params}")


# Prophet

In [15]:
import itertools
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
import holidays

# Function to calculate sMAPE
def smape(a, f):
    return 100 * np.mean(2 * np.abs(f - a) / (np.abs(a) + np.abs(f)))

# Assuming data is already defined with 'ds' and 'y' columns
# Add is_public_holiday column
holiday = holidays.CountryHoliday('UK')
data['is_public_holiday'] = data['ds'].apply(lambda date: 1 if date in holiday else 0)

# Define hyperparameters to tune
param_grid = {
    'changepoint_prior_scale': [0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.1, 1.0, 10.0],
    'holidays_prior_scale': [0.1, 1.0, 10.0],
    'seasonality_mode': ['additive', 'multiplicative']
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

best_params = None
best_mape = float('inf')
best_mae = float('inf')

# Grid search to find the best hyperparameters
for params in all_params:
    model = Prophet(**params)
    model.add_regressor('is_public_holiday')
    model.fit(data)

    # Cross-validate the model
    df_cv = cross_validation(model, initial='547 days', period='180 days', horizon='30 days')
    df_p = performance_metrics(df_cv)
    
    # Calculate metrics
    mape = mean_absolute_percentage_error(df_cv['y'], df_cv['yhat'])
    mae = mean_absolute_error(df_cv['y'], df_cv['yhat'])
    
    if mape < best_mape:
        best_mape = mape
        best_mae = mae
        best_params = params

# Output the best parameters and metrics
print(f"Best Parameters: {best_params}")
print(f"Best MAPE: {round(best_mape, 2)}")
print(f"Best MAE: {round(best_mae, 2)}")

# Train the final model with the best parameters
model = Prophet(**best_params)
model.add_regressor('is_public_holiday')
model.fit(data)

# Forecast future values for the next 30 days
future = model.make_future_dataframe(periods=30)
future['is_public_holiday'] = future['ds'].apply(lambda date: 1 if date in holiday else 0)

forecast = model.predict(future)

# Visualize Results
model.plot(forecast)
model.plot_components(forecast)


In [17]:
import itertools
from sklearn.metrics import mean_absolute_percentage_error
import pandas as pd
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_cross_validation_metric
import holidays


# Define hyperparameters to tune
param_grid = {  
    'changepoint_prior_scale': [0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.1, 1.0, 10.0],
    'holidays_prior_scale': [0.1, 1.0, 10.0],
    'seasonality_mode': ['additive', 'multiplicative']
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]

best_params = None
best_mape = float('inf')

# Grid search to find the best hyperparameters
for params in all_params:
    model = Prophet(**params)
    model.add_regressor('is_public_holiday')
    model.fit(data)

    # Cross-validate the model
    df_cv = cross_validation(model, initial='547 days', period='180 days', horizon='30 days')
    df_p = performance_metrics(df_cv)
    
    # Calculate MAPE
    mape = mean_absolute_percentage_error(df_cv['y'], df_cv['yhat'])
    
    if mape < best_mape:
        best_mape = mape
        best_params = params

# Output the best parameters and MAPE
print(f"Best Parameters: {best_params}")
print(f"Best MAPE: {round(best_mape, 2)}")

# Train the final model with the best parameters
model = Prophet(**best_params)
model.add_regressor('is_public_holiday')
model.fit(data)

# Forecast future values
future = model.make_future_dataframe(periods=365)
future['is_public_holiday'] = future['ds'].apply(
    lambda date: 1 if date in holiday else 0
)

forecast = model.predict(future)

# Visualize Results
model.plot(forecast)
model.plot_components(forecast)

# Evaluate Accuracy
df_cv = cross_validation(model, initial='547 days', period='180 days', horizon='30 days')
df_p = performance_metrics(df_cv)
print(df_p.head().round(2))

fig = plot_cross_validation_metric(df_cv, metric='rmse')

# Calculate MAPE using yhat and y
mape = mean_absolute_percentage_error(df_cv['y'], df_cv['yhat'])
print(f"MAPE: {round(mape, 2)}")
